In [0]:
checkpoint_path = spark.sql("""DESCRIBE EXTERNAL LOCATION `checkpoints`""").select('url').collect()[0][0]

In [0]:
landing_path = spark.sql("""DESCRIBE EXTERNAL LOCATION `landing`""").select('url').collect()[0][0]

In [0]:
print(landing_path)

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql.functions import current_timestamp
schema = StructType([
    StructField("Record_ID",IntegerType()),
    StructField("Count_point_id",IntegerType()),
    StructField("Direction_of_travel",StringType()),
    StructField("Year",IntegerType()),
    StructField("Count_date",StringType()),
    StructField("hour",IntegerType()),
    StructField("Region_id",IntegerType()),
    StructField("Region_name",StringType()),
    StructField("Local_authority_name",StringType()),
    StructField("Road_name",StringType()),
    StructField("Road_Category_ID",IntegerType()),
    StructField("Start_junction_road_name",StringType()),
    StructField("End_junction_road_name",StringType()),
    StructField("Latitude",DoubleType()),
    StructField("Longitude",DoubleType()),
    StructField("Link_length_km",DoubleType()),
    StructField("Pedal_cycles",IntegerType()),
    StructField("Two_wheeled_motor_vehicles",IntegerType()),
    StructField("Cars_and_taxis",IntegerType()),
    StructField("Buses_and_coaches",IntegerType()),
    StructField("LGV_Type",IntegerType()),
    StructField("HGV_Type",IntegerType()),
    StructField("EV_Car",IntegerType()),
    StructField("EV_Bike",IntegerType())
    ])

In [0]:
from pyspark.sql.functions import current_timestamp
raw_traffic_stream = spark.readStream.format('cloudFiles').option('cloudFiles.format','csv').option('cloudFiles.schemaLocation',f'{checkpoint_path}/rawTrafficLoad/schemaInfer').option('header','true').schema(schema).load(landing_path+'/raw_traffic/').withColumn('extract_time',current_timestamp())

In [0]:
raw_traffic_stream.printSchema()

In [0]:
raw_traffic_stream.writeStream.format('delta').option('checkpointLocation',f'{checkpoint_path}/rawTrafficLoad/Checkpt').outputMode('append').queryName('rawTrafficWriteStream').trigger(availableNow=True).toTable("`dev_catalog`.`bronze`.`raw_traffic`")

In [0]:
%sql
select count(*) from dev_catalog.bronze.raw_traffic;

### Raw_road data to bronze

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
schema = StructType([
     

    StructField('Road_ID',IntegerType()),

    StructField('Road_Category_Id',IntegerType()),

    StructField('Road_Category',StringType()),

    StructField('Region_ID',IntegerType()),

    StructField('Region_Name',StringType()),

    StructField('Total_Link_Length_Km',DoubleType()),

    StructField('Total_Link_Length_Miles',DoubleType()),

    StructField('All_Motor_Vehicles',DoubleType())
])

In [0]:
raw_road_stream = spark.readStream.format('cloudFiles').option('cloudFiles.format','csv').option('cloudFiles.schemaLocation',f'{checkpoint_path}/rawRoadLoad/schemaInfer').option('header','true').schema(schema).load(landing_path+'/raw_roads/')

In [0]:
raw_road_stream.writeStream.format('delta').option('checkpointLocation',f'{checkpoint_path}/rawRoadLoad/Checkpt').outputMode('append').queryName('rawRoadWriteStream').trigger(availableNow=True).toTable("`dev_catalog`.`bronze`.`raw_roads`")

In [0]:
%sql
select * from dev_catalog.bronze.raw_roads;